## Weather Data : Visualization After Smoothing

### For New York State

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
%pylab inline

import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStatistics.py'])

from pyspark import SparkContext
from pyspark.sql import *
import pyspark.sql
sqlContext = SQLContext(sc)

import numpy as np
from lib.numpy_pack import packArray,unpackArray
from lib.spark_PCA import computeCov
from lib.computeStatistics import *

Populating the interactive namespace from numpy and matplotlib


In [2]:
import warnings  # Suppress Warnings
warnings.filterwarnings('ignore')

_figsize=(10,7)

## Read Data

### Read Raw data for state

In [34]:
state='NE'
data_dir='../Data/Weather'

tarname=state+'.tgz'
parquet=state+'.parquet'

In [35]:
!rm -rf $data_dir/$tarname

command="curl https://mas-dse-open.s3.amazonaws.com/Weather/by_state_2/%s > %s/%s"%(tarname,data_dir,tarname)
print(command)
!$command
!ls -lh $data_dir/$tarname


curl https://mas-dse-open.s3.amazonaws.com/Weather/by_state_2/NE.tgz > ../Data/Weather/NE.tgz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 63.6M  100 63.6M    0     0   923k      0  0:01:10  0:01:10 --:--:-- 1643k0   772k      0  0:01:24  0:00:58  0:00:26 1683k
-rw-r--r-- 1 jovyan users 64M May 14 01:57 ../Data/Weather/NE.tgz


In [ ]:
def get_data(state):
    
    !ls -l $data_dir
    
    cur_dir,=!pwd
    %cd $data_dir
    !tar -xzf $tarname
    !du -h ./$parquet
    %cd $cur_dir
    
    !du -h $data_dir/$parquet
    
    print(parquet)
    weather_df=sqlContext.read.parquet(data_dir+'/'+parquet)
    
    sqlContext.registerDataFrameAsTable(weather_df,'weather_df')
    
    # get all measurements for years 1970-2010
    year_low = 1970
    year_high = 2010
    measurement = 'TMIN'
    Query="""
    SELECT *
    FROM weather_df
    WHERE Year >='%d' 
    and Year <='%d'
    and Measurement='%s' 
    """%(year_low, year_high, measurement )

    sqlContext.sql(Query).show()

    pandas_df=sqlContext.sql(Query).toPandas()
    pandas_df=pandas_df.set_index('Measurement')
    pandas_df
    
    # Only keep stations which have values for each year in 1970-2010. 
    pandas_df = pandas_df[pandas_df['name'].isin(pandas_df['name'].value_counts()[pandas_df['name'].value_counts()==41].index)]
    
    pandas_df['TEMP_Means'] = list(pd.DataFrame(pandas_df['Values'].values.tolist()).mean(1))

    year_means = pd.DataFrame(pandas_df.groupby('Year')['TEMP_Means'].mean())
    
    return pandas_df, year_means
    

In [ ]:
weather_df, pandas_df = get_data(state)


In [ ]:
MN_weather_df.to_csv("MN_TMIN_weather_df.csv")
MN_year_means.to_csv("MN_TMIN_year_means_df.csv")

In [4]:
!ls -l $data_dir

total 442668
drwxr-xr-x 10 jovyan users       320 May  4 03:52 decon_NY_SNWD.parquet
drwxr-xr-x 31 jovyan users       992 Apr 19  2018 KS.parquet
-rw-r--r--  1 jovyan users  60682406 May 14 00:51 KS.tgz
drwxr-xr-x 30 jovyan users       960 Apr 19  2018 MN.parquet
-rw-r--r--  1 jovyan users  60550692 May 14 01:49 MN.tgz
drwxr-xr-x 31 jovyan users       992 Apr 19  2018 ND.parquet
-rw-r--r--  1 jovyan users  51874382 May 14 01:38 ND.tgz
drwxr-xr-x 31 jovyan users       992 Apr 19  2018 OK.parquet
-rw-r--r--  1 jovyan users  50981500 May 14 01:34 OK.tgz
drwxr-xr-x 10 jovyan users       320 May  4 03:52 recon_NY_SNWD.parquet
drwxr-xr-x 31 jovyan users       992 Apr 19  2018 SD.parquet
-rw-r--r--  1 jovyan users  57078283 May 14 01:41 SD.tgz
drwxr-xr-x 10 jovyan users       320 May  4 03:52 stations.parquet
-rw-r--r--  1 jovyan users  51365694 May  4 03:52 STAT_NY.pickle
drwxr-xr-x 31 jovyan users       992 Apr 19  2018 TX.parquet
-rw-r--r--  1 jovyan users 114555787 May 14 01:28 TX.tgz
-rw

In [5]:
cur_dir,=!pwd
%cd $data_dir
!tar -xzf $tarname
!du -h ./$parquet
%cd $cur_dir

/home/jovyan/work/Sections/Data/Weather
70M	./MN.parquet
/home/jovyan/work/Sections/Section2-Weather-PCA


In [6]:
!du -h $data_dir/$parquet

70M	../Data/Weather/MN.parquet


In [7]:
print(parquet)
weather_df=sqlContext.read.parquet(data_dir+'/'+parquet)

MN.parquet


In [8]:
sqlContext.registerDataFrameAsTable(weather_df,'weather_df')

In [26]:
# get all measurements for years 1970-2010
year_low = 1970
year_high = 2010
measurement = 'TMIN'
Query="""
SELECT *
FROM weather_df
WHERE Year >='%d' 
and Year <='%d'
and Measurement='%s' 
"""%(year_low, year_high, measurement )

sqlContext.sql(Query).show()

pandas_df=sqlContext.sql(Query).toPandas()
pandas_df=pandas_df.set_index('Measurement')
pandas_df

+-----------+-----------+----+--------------------+---------------+------------------+------------------+------------------+-----+------+
|    Station|Measurement|Year|              Values|     dist_coast|          latitude|         longitude|         elevation|state|  name|
+-----------+-----------+----+--------------------+---------------+------------------+------------------+------------------+-----+------+
|USC00215392|       TMIN|1970|[90 D5 B0 D8 10 D...|1138.2900390625|45.753299713134766|-93.66169738769531|324.29998779296875|   MN|MILACA|
|USC00215392|       TMIN|1971|[80 D4 F0 D6 60 D...|1138.2900390625|45.753299713134766|-93.66169738769531|324.29998779296875|   MN|MILACA|
|USC00215392|       TMIN|1972|[30 D5 E0 D8 C8 D...|1138.2900390625|45.753299713134766|-93.66169738769531|324.29998779296875|   MN|MILACA|
|USC00215392|       TMIN|1973|[E0 D8 38 D9 E0 D...|1138.2900390625|45.753299713134766|-93.66169738769531|324.29998779296875|   MN|MILACA|
|USC00215392|       TMIN|1974|[90 

,Station,Year,Values,dist_coast,latitude,longitude,elevation,state,name
Measurement,,,,,,,,,
TMIN,USC00215392,1970,"[144, 213, 176, 216, 16, 218, 16, 218, 88, 220...",1138.290039,45.753300,-93.661697,324.299988,MN,MILACA
TMIN,USC00215392,1971,"[128, 212, 240, 214, 96, 217, 96, 217, 160, 21...",1138.290039,45.753300,-93.661697,324.299988,MN,MILACA
TMIN,USC00215392,1972,"[48, 213, 224, 216, 200, 218, 64, 220, 200, 21...",1138.290039,45.753300,-93.661697,324.299988,MN,MILACA
TMIN,USC00215392,1973,"[224, 216, 56, 217, 224, 212, 72, 219, 152, 22...",1138.290039,45.753300,-93.661697,324.299988,MN,MILACA
TMIN,USC00215392,1974,"[144, 221, 52, 221, 20, 220, 72, 219, 32, 219,...",1138.290039,45.753300,-93.661697,324.299988,MN,MILACA
TMIN,USC00215392,1975,"[88, 216, 128, 216, 224, 212, 96, 217, 184, 21...",1138.290039,45.753300,-93.661697,324.299988,MN,MILACA
TMIN,USC00215392,1976,"[64, 210, 160, 215, 32, 219, 176, 220, 20, 220...",1138.290039,45.753300,-93.661697,324.299988,MN,MILACA
TMIN,USC00215392,1977,"[112, 218, 176, 216, 88, 216, 72, 219, 232, 21...",1138.290039,45.753300,-93.661697,324.299988,MN,MILACA
TMIN,USC00215392,1978,"[0, 126, 0, 126, 0, 126, 0, 126, 0, 126, 0, 12...",1138.290039,45.753300,-93.661697,324.299988,MN,MILACA


In [27]:
# Only keep stations which have values for each year in 1970-2010. 
pandas_df = pandas_df[pandas_df['name'].isin(pandas_df['name'].value_counts()[pandas_df['name'].value_counts()==41].index)]

In [28]:
#pandas_df.to_csv("TX_TMIN_weather_df.csv")

In [29]:
pandas_df['TEMP_Means'] = list(pd.DataFrame(pandas_df['Values'].values.tolist()).mean(1))


In [30]:

year_means = pd.DataFrame(pandas_df.groupby('Year')['TEMP_Means'].mean())

In [31]:
year_means

,TEMP_Means
Year,
1970,124.607032
1971,123.991196
1972,124.540822
1973,120.764438
1974,125.792000
1975,124.726009
1976,125.978484
1977,121.155543
1978,124.488329


In [32]:
MN_weather_df = pandas_df
MN_year_means = year_means

In [33]:
#ND_weather_df = pandas_df
#ND_year_means = year_means

In [51]:
#SD_weather_df = pandas_df
#SD_year_means = year_means

In [33]:
MN_weather_df.to_csv("MN_TMIN_weather_df.csv")
MN_year_means.to_csv("MN_TMIN_year_means_df.csv")